In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)
pd.set_option('float_format', '{:f}'.format)
import warnings
warnings.filterwarnings("ignore")

In [2]:
# DIAG = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\DIAG_2l.txt.gz",sep="|")

#Procedure data
PX = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\PX_2l.txt.gz",sep="|")

# Presecription data
# RX = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\RX_2l.txt.gz",sep="|")

# Diagnosis Code reference
BC_SN = pd.read_excel(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\BC_SN ICD Code.xlsx")

In [3]:
# Get procedure information
Procedure_refer = pd.read_csv(r"C:\Users\elena\Downloads\OneDrive_1_10-4-2020\APLD Raw Data\PROCEDURE_reference_20200925.txt",sep='|')
PX_merged = PX.merge(Procedure_refer,how="left",on="PROCEDURE_CODE").merge(BC_SN,how="left", left_on='DIAGNOSIS_CODE',right_on ='diagnosis_cd').drop(columns=['diagnosis_cd','DIAGNOSIS_CODE','version','sub_indication_cd'])
PX_merged = PX_merged[['PATIENT_ID','SERVICE_DATE','indication_cd','PROCEDURE_CODE','PROCEDURE_DESCRIPTION','SVC_CRGD_AMT','UNIT_OF_SVC_AMT','PRC_DOSAGE_AMT','PLACE_OF_SERVICE','PAY_TYPE']]

In [4]:
# Dealing with the service date
def lookup(s):
    """ 
    This is a faster approach to convert string to datetime object
    """
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.apply(lambda v: dates[v])

PX_merged['SERVICE_DATE'] = lookup(PX_merged['SERVICE_DATE'])
# RX_merged['SERVICE_DATE'] = lookup(RX_merged['SERVICE_DATE'])

In [5]:
def time_feature(df):
    min_dates = df.groupby(['PATIENT_ID'])['SERVICE_DATE'].min()
    df['Earliest_DIAG_Date'] = df.apply(lambda row: min_dates.loc[row['PATIENT_ID']], axis=1)

    df['Days_Diff'] = df['SERVICE_DATE'] - df['Earliest_DIAG_Date']
    df['Week_Diff'] = df['Days_Diff']/np.timedelta64(1,"W")
    df['Month_Diff'] = df['Days_Diff']/np.timedelta64(1,"M")
    df['Year_Diff'] = df['Days_Diff']/np.timedelta64(1,'Y')
    df['Days_Diff'] = df['Days_Diff'].dt.days

    return df

In [6]:
# Get time_features
PX_merged = time_feature(PX_merged)

In [7]:
# Processing with the procedure type
def PRC_type(df):
    df['CHEMOTHERAPY'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("CHEMOTHERAPY"),1,0)
    df['CYCLOPHOSPHAMIDE'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("CYCLOPHOSPHAMIDE"),1,0)
    df['LYMPH'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("LYMPH"),1,0)
    df['GENE'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("GENE"),1,0)
    df['IMMUNOHISTOCHEMISTRY'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("IMMUNOHISTOCHEMISTRY"),1,0)
    df['IMMUNOCYTOCHEMISTRY'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("IMMUNOCYTOCHEMISTRY"),1,0)
    df['IMMUNOASSAY'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("IMMUNOASSAY"),1,0)
    df['METASTATIC'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("METASTATIC"),1,0)
    df['SURGI'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("SURGI"),1,0)
    df['MASTECTOMY'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("MASTECTOMY"),1,0)
    df['SUCCINYLCHOLINE CHLORIDE'] = np.where(df['PROCEDURE_DESCRIPTION'].str.contains("SUCCINYLCHOLINE CHLORIDE"),1,0)
    return df

PX_merged = PRC_type(PX_merged)

In [8]:
PX_merged = PX_merged.sort_values(['PATIENT_ID','SERVICE_DATE'],ascending = (True, True))

In [9]:
# Get the dosage amount
PX_merged['PRC_DOSAGE_AMT'] = PX_merged.PRC_DOSAGE_AMT.fillna(0)
AMT =  pd.DataFrame(PX_merged.groupby(['PATIENT_ID','SERVICE_DATE','PROCEDURE_CODE'])[['SVC_CRGD_AMT','UNIT_OF_SVC_AMT',"PRC_DOSAGE_AMT"]].sum())
AMT.reset_index(inplace=True)
PX_merged = PX_merged.drop(columns=['PROCEDURE_DESCRIPTION','SVC_CRGD_AMT','UNIT_OF_SVC_AMT','PRC_DOSAGE_AMT'])
PX_merged = PX_merged.merge(AMT,how='left',on=['PATIENT_ID','SERVICE_DATE','PROCEDURE_CODE'])

In [10]:
# Get the place dummies and pay type dummies 
PX_merged['PLACE_OF_SERVICE'] = PX_merged['PLACE_OF_SERVICE'].replace({np.nan: 'NONE'})
PX_merged['PAY_TYPE'] = PX_merged['PAY_TYPE'].replace({np.nan:'NONE'})

Place_dummies = pd.get_dummies(PX_merged.PLACE_OF_SERVICE, prefix='Place')
PAY_TYPE_dummies = pd.get_dummies(PX_merged.PAY_TYPE,prefix='PAY_TYPE')

Place_dummies.reset_index(inplace=True)
PAY_TYPE_dummies.reset_index(inplace=True)

PX_merged.reset_index(inplace=True)

In [11]:
PX_merged.head()

,index,PATIENT_ID,SERVICE_DATE,indication_cd,PROCEDURE_CODE,PLACE_OF_SERVICE,PAY_TYPE,Earliest_DIAG_Date,Days_Diff,Week_Diff,Month_Diff,Year_Diff,CHEMOTHERAPY,CYCLOPHOSPHAMIDE,LYMPH,GENE,IMMUNOHISTOCHEMISTRY,IMMUNOCYTOCHEMISTRY,IMMUNOASSAY,METASTATIC,SURGI,MASTECTOMY,SUCCINYLCHOLINE CHLORIDE,SVC_CRGD_AMT,UNIT_OF_SVC_AMT,PRC_DOSAGE_AMT
0,0,29360529,2013-09-20,BC,36415,NONE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,40.000000,2.000000,0.000000
1,1,29360529,2013-09-20,BC,85025,NONE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,72.000000,2.000000,0.000000
2,2,29360529,2013-09-20,BC,99214,NONE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,462.000000,2.000000,0.000000
3,3,29360529,2013-09-20,BC,36415,OFFICE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,40.000000,2.000000,0.000000
4,4,29360529,2013-09-20,BC,85025,OFFICE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,72.000000,2.000000,0.000000


In [13]:
# Get number of procedure per patient per day
PX_count = pd.DataFrame(PX_merged.groupby(['PATIENT_ID','SERVICE_DATE']).size())
PX_count.reset_index(inplace=True)
PX_count = PX_count.rename(columns={0:"PX_count"})

In [14]:
PX_count.head()

,PATIENT_ID,SERVICE_DATE,PX_count
0,29360529,2013-09-20,6
1,29360529,2013-10-11,2
2,29360529,2013-12-20,8
3,29360529,2014-01-13,1
4,29360529,2014-01-17,1


In [15]:
# Decide whether the procedure is severe, light or in-between
PX_merged['PX_in-between'] = np.where(PX_merged.loc[:,'CHEMOTHERAPY':'SUCCINYLCHOLINE CHLORIDE'].sum(axis=1)==0,1,0)
PX_merged['PX_Severe'] = np.where(PX_merged.loc[:,'CHEMOTHERAPY':'METASTATIC'].sum(axis=1)!=0,1,0)
PX_merged['PX_light'] = np.where(((PX_merged['PX_in-between']==0) & (PX_merged['PX_Severe']==0)),1,0)

In [17]:
PX_merged = PX_merged.merge(PX_count,how="left",on=['PATIENT_ID','SERVICE_DATE'])

In [18]:
PX_temp = PX_merged[list(PX_merged.loc[:,'PATIENT_ID':'SERVICE_DATE']) + list(PX_merged.loc[:,'CHEMOTHERAPY':'PX_light'])]
PX_temp = pd.DataFrame(PX_temp.groupby(['PATIENT_ID','SERVICE_DATE']).sum())

In [21]:
PX_merged.head()

,index,PATIENT_ID,SERVICE_DATE,indication_cd,PROCEDURE_CODE,PLACE_OF_SERVICE,PAY_TYPE,Earliest_DIAG_Date,Days_Diff,Week_Diff,Month_Diff,Year_Diff,CHEMOTHERAPY,CYCLOPHOSPHAMIDE,LYMPH,GENE,IMMUNOHISTOCHEMISTRY,IMMUNOCYTOCHEMISTRY,IMMUNOASSAY,METASTATIC,SURGI,MASTECTOMY,SUCCINYLCHOLINE CHLORIDE,SVC_CRGD_AMT,UNIT_OF_SVC_AMT,PRC_DOSAGE_AMT,PX_in-between,PX_Severe,PX_light,PX_count
0,0,29360529,2013-09-20,BC,36415,NONE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,40.000000,2.000000,0.000000,1,0,0,6
1,1,29360529,2013-09-20,BC,85025,NONE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,72.000000,2.000000,0.000000,1,0,0,6
2,2,29360529,2013-09-20,BC,99214,NONE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,462.000000,2.000000,0.000000,1,0,0,6
3,3,29360529,2013-09-20,BC,36415,OFFICE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,40.000000,2.000000,0.000000,1,0,0,6
4,4,29360529,2013-09-20,BC,85025,OFFICE,THIRD PARTY,2013-09-20,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,72.000000,2.000000,0.000000,1,0,0,6


In [22]:
PX_temp1 = PX_merged[['PATIENT_ID','SERVICE_DATE','indication_cd','Earliest_DIAG_Date','Days_Diff','Week_Diff','Year_Diff']]
PX_final = PX_temp1.merge(PX_temp,how="left",on=['PATIENT_ID','SERVICE_DATE'])
PX_final = PX_final.drop_duplicates(subset=['PATIENT_ID','SERVICE_DATE'])

In [24]:
PX_final.shape

(1801344, 24)

In [26]:
PX_final.to_pickle("./PX_final.pkl")